# Using roboflow dataset to detect people using MediaPipe

In [ ]:
#!pip install 'keras<3.0.0' mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import libraries

In [ ]:
import os
import tensorflow as tf
#assert tf.__version__.startswith('2')
#from google.colab import files

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Download dataset directly from Roboflow

In [ ]:
#!pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="rda3pff5KNqzUs0cUoSA")
# project = rf.workspace("yolodataset").project("person-dataset-mvbk4")
# version = project.version(1)
# dataset = version.download("coco")


## Creating the steps for training

In [ ]:
class TransferLearning():
    """ Transfer Learning class for object detection using MediaPipe Model Maker
    2 datasets are required: train and validation
    
    Attributes:
        train_dataset_path: path to the train dataset
        validation_dataset_path: path to the validation dataset
        model: model to be trained
        train_data: train dataset
        validation_data: validation dataset
    """
    def __init__(self):
        self.train_dataset_path = "/content/drive/MyDrive/PersonD/train"
        self.validation_dataset_path = "/content/drive/MyDrive/PersonD/validate" 
        self.model = None
        self.train_data = None
        self.val_data = None
        self.hparams = None
        self.options = None
        
    def load_data(self):
        """ Load train and validation datasets from the given paths        """
        self.train_data = object_detector.DataLoader.from_pascal_voc(self.train_dataset_path)
        self.validation_data = object_detector.DataLoader.from_pascal_voc(self.validation_dataset_path)
    
    def train_model(self,batch_size=8, learning_rate=0.3, epochs=50, export_dir='exported_model'):
        """ Train the model using the loaded datasets
        
        Args:
            batch_size: batch size for training
            learning_rate: learning rate for training
            epochs: number of epochs for training
            export_dir: directory to export the trained model
        """
        self.hparams = object_detector.HParams(batch_size, learning_rate, epochs, export_dir)
        self.options = object_detector.ObjectDetectorOptions(
            supported_model=object_detector.SupportedModels.MOBILENET_V2,
            hparams=self.hparams
        )
        
        self.model = object_detector.ObjectDetector.create(
            train_data=self.train_data,
            validation_data=self.val_data,
            options=self.options)

    def evaluate_model(self,batch_size=8):
        """ Evaluate the trained model 
        
        Args:
            batch_size: batch size for evaluation
        """
        loss, metrics = self.model.evaluate(self.val_data,batch_size)
        print(f"Validation loss: {loss}")
        print(f"Validation coco metrics: {metrics}")
    
    def export_model(self):
        """ Export the trained model """
        self.model.export('people-detection.tflite')
        print(f"Model exported")
    
    




## Loading the data, and training

In [ ]:
TransferLearning_model = TransferLearning()
TransferLearning_model.load_data()
TransferLearning_model.train_model()


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   149056    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


36/36 [==============================] - 73s 439ms/step - total_loss: 5.5539 - cls_loss: 5.0689 - box_loss: 0.0086 - model_loss: 5.4989 - val_total_loss: 0.9363 - val_cls_loss: 0.6694 - val_box_loss: 0.0042 - val_model_loss: 0.8813
Epoch 2/50
36/36 [==============================] - 7s 188ms/step - total_loss: 1.4140 - cls_loss: 0.9963 - box_loss: 0.0073 - model_loss: 1.3590 - val_total_loss: 0.8953 - val_cls_loss: 0.5793 - val_box_loss: 0.0052 - val_model_loss: 0.8404
Epoch 3/50
36/36 [==============================] - 9s 251ms/step - total_loss: 1.0666 - cls_loss: 0.7319 - box_loss: 0.0056 - model_loss: 1.0116 - val_total_loss: 0.7628 - val_cls_loss: 0.4543 - val_box_loss: 0.0051 - val_model_loss: 0.7078
Epoch 4/50
36/36 [==============================] - 7s 195ms/step - total_loss: 0.7923 - cls_loss: 0.5153 - box_loss: 0.0044 - model_loss: 0.7373 - val_total_loss: 0.7328 - val_cls_loss: 0.4268 - val_box_loss: 0.0050 - val_model_loss: 0.6778
Epoch 5/50
36/36 [========================

## Evaluating the model 

In [ ]:
TransferLearning_model.evaluate_model()

55/55 [==============================] - 4s 42ms/step - total_loss: 0.8395 - cls_loss: 0.4977 - box_loss: 0.0057 - model_loss: 0.7845
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.12s).
Accumulating evaluation results...
DONE (t=0.20s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.240
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.022
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.109
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.109
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.199
 Average Recall     (AR) @[ I

In [ ]:
TransferLearning_model.export_model()


## Plotting all loss values using matplotlib

In [ ]:
import re
import matplotlib.pyplot as plt
txt_file = """
Epoch 1/50
36/36 [==============================] - 73s 439ms/step - total_loss: 5.5539 - cls_loss: 5.0689 - box_loss: 0.0086 - model_loss: 5.4989 - val_total_loss: 0.9363 - val_cls_loss: 0.6694 - val_box_loss: 0.0042 - val_model_loss: 0.8813
Epoch 2/50
36/36 [==============================] - 7s 188ms/step - total_loss: 1.4140 - cls_loss: 0.9963 - box_loss: 0.0073 - model_loss: 1.3590 - val_total_loss: 0.8953 - val_cls_loss: 0.5793 - val_box_loss: 0.0052 - val_model_loss: 0.8404
Epoch 3/50
36/36 [==============================] - 9s 251ms/step - total_loss: 1.0666 - cls_loss: 0.7319 - box_loss: 0.0056 - model_loss: 1.0116 - val_total_loss: 0.7628 - val_cls_loss: 0.4543 - val_box_loss: 0.0051 - val_model_loss: 0.7078
Epoch 4/50
36/36 [==============================] - 7s 195ms/step - total_loss: 0.7923 - cls_loss: 0.5153 - box_loss: 0.0044 - model_loss: 0.7373 - val_total_loss: 0.7328 - val_cls_loss: 0.4268 - val_box_loss: 0.0050 - val_model_loss: 0.6778
Epoch 5/50
36/36 [==============================] - 9s 248ms/step - total_loss: 0.6459 - cls_loss: 0.4111 - box_loss: 0.0036 - model_loss: 0.5909 - val_total_loss: 0.6905 - val_cls_loss: 0.3907 - val_box_loss: 0.0049 - val_model_loss: 0.6355
Epoch 6/50
36/36 [==============================] - 9s 239ms/step - total_loss: 0.5366 - cls_loss: 0.3336 - box_loss: 0.0030 - model_loss: 0.4816 - val_total_loss: 0.7232 - val_cls_loss: 0.4060 - val_box_loss: 0.0052 - val_model_loss: 0.6682
Epoch 7/50
36/36 [==============================] - 7s 183ms/step - total_loss: 0.4647 - cls_loss: 0.2861 - box_loss: 0.0025 - model_loss: 0.4097 - val_total_loss: 0.6277 - val_cls_loss: 0.3230 - val_box_loss: 0.0050 - val_model_loss: 0.5727
Epoch 8/50
36/36 [==============================] - 7s 184ms/step - total_loss: 0.4246 - cls_loss: 0.2553 - box_loss: 0.0023 - model_loss: 0.3695 - val_total_loss: 0.6450 - val_cls_loss: 0.3381 - val_box_loss: 0.0050 - val_model_loss: 0.5900
Epoch 9/50
36/36 [==============================] - 7s 194ms/step - total_loss: 0.3744 - cls_loss: 0.2220 - box_loss: 0.0019 - model_loss: 0.3193 - val_total_loss: 0.6304 - val_cls_loss: 0.3183 - val_box_loss: 0.0051 - val_model_loss: 0.5754
Epoch 10/50
36/36 [==============================] - 9s 245ms/step - total_loss: 0.3322 - cls_loss: 0.1918 - box_loss: 0.0017 - model_loss: 0.2772 - val_total_loss: 0.6299 - val_cls_loss: 0.3126 - val_box_loss: 0.0052 - val_model_loss: 0.5748
Epoch 11/50
36/36 [==============================] - 8s 225ms/step - total_loss: 0.3027 - cls_loss: 0.1724 - box_loss: 0.0015 - model_loss: 0.2476 - val_total_loss: 0.6212 - val_cls_loss: 0.3095 - val_box_loss: 0.0051 - val_model_loss: 0.5662
Epoch 12/50
36/36 [==============================] - 8s 216ms/step - total_loss: 0.2860 - cls_loss: 0.1590 - box_loss: 0.0014 - model_loss: 0.2310 - val_total_loss: 0.6604 - val_cls_loss: 0.3332 - val_box_loss: 0.0054 - val_model_loss: 0.6053
Epoch 13/50
36/36 [==============================] - 9s 249ms/step - total_loss: 0.2867 - cls_loss: 0.1623 - box_loss: 0.0014 - model_loss: 0.2316 - val_total_loss: 0.6459 - val_cls_loss: 0.3168 - val_box_loss: 0.0055 - val_model_loss: 0.5908
Epoch 14/50
36/36 [==============================] - 7s 189ms/step - total_loss: 0.2719 - cls_loss: 0.1500 - box_loss: 0.0013 - model_loss: 0.2169 - val_total_loss: 0.6581 - val_cls_loss: 0.3326 - val_box_loss: 0.0054 - val_model_loss: 0.6031
Epoch 15/50
36/36 [==============================] - 8s 213ms/step - total_loss: 0.2637 - cls_loss: 0.1426 - box_loss: 0.0013 - model_loss: 0.2086 - val_total_loss: 0.6696 - val_cls_loss: 0.3476 - val_box_loss: 0.0053 - val_model_loss: 0.6145
Epoch 16/50
36/36 [==============================] - 7s 191ms/step - total_loss: 0.2635 - cls_loss: 0.1430 - box_loss: 0.0013 - model_loss: 0.2084 - val_total_loss: 0.6855 - val_cls_loss: 0.3601 - val_box_loss: 0.0054 - val_model_loss: 0.6304
Epoch 17/50
36/36 [==============================] - 9s 247ms/step - total_loss: 0.2324 - cls_loss: 0.1228 - box_loss: 0.0011 - model_loss: 0.1774 - val_total_loss: 0.6480 - val_cls_loss: 0.3302 - val_box_loss: 0.0053 - val_model_loss: 0.5929
Epoch 18/50
36/36 [==============================] - 9s 252ms/step - total_loss: 0.2134 - cls_loss: 0.1092 - box_loss: 9.8348e-04 - model_loss: 0.1583 - val_total_loss: 0.6621 - val_cls_loss: 0.3383 - val_box_loss: 0.0054 - val_model_loss: 0.6071
Epoch 19/50
36/36 [==============================] - 7s 188ms/step - total_loss: 0.2082 - cls_loss: 0.1064 - box_loss: 9.3268e-04 - model_loss: 0.1531 - val_total_loss: 0.6903 - val_cls_loss: 0.3633 - val_box_loss: 0.0054 - val_model_loss: 0.6352
Epoch 20/50
36/36 [==============================] - 8s 214ms/step - total_loss: 0.2218 - cls_loss: 0.1132 - box_loss: 0.0011 - model_loss: 0.1667 - val_total_loss: 0.6749 - val_cls_loss: 0.3500 - val_box_loss: 0.0054 - val_model_loss: 0.6198
Epoch 21/50
36/36 [==============================] - 8s 219ms/step - total_loss: 0.1997 - cls_loss: 0.0994 - box_loss: 9.0553e-04 - model_loss: 0.1446 - val_total_loss: 0.7200 - val_cls_loss: 0.3815 - val_box_loss: 0.0057 - val_model_loss: 0.6649
Epoch 22/50
36/36 [==============================] - 8s 223ms/step - total_loss: 0.1840 - cls_loss: 0.0892 - box_loss: 7.9451e-04 - model_loss: 0.1289 - val_total_loss: 0.6921 - val_cls_loss: 0.3703 - val_box_loss: 0.0053 - val_model_loss: 0.6371
Epoch 23/50
36/36 [==============================] - 8s 216ms/step - total_loss: 0.1805 - cls_loss: 0.0864 - box_loss: 7.8160e-04 - model_loss: 0.1254 - val_total_loss: 0.7082 - val_cls_loss: 0.3831 - val_box_loss: 0.0054 - val_model_loss: 0.6531
Epoch 24/50
36/36 [==============================] - 9s 248ms/step - total_loss: 0.1857 - cls_loss: 0.0869 - box_loss: 8.7382e-04 - model_loss: 0.1306 - val_total_loss: 0.6867 - val_cls_loss: 0.3708 - val_box_loss: 0.0052 - val_model_loss: 0.6316
Epoch 25/50
36/36 [==============================] - 7s 187ms/step - total_loss: 0.1765 - cls_loss: 0.0827 - box_loss: 7.7420e-04 - model_loss: 0.1214 - val_total_loss: 0.7415 - val_cls_loss: 0.4111 - val_box_loss: 0.0055 - val_model_loss: 0.6864
Epoch 26/50
36/36 [==============================] - 10s 289ms/step - total_loss: 0.1729 - cls_loss: 0.0803 - box_loss: 7.5035e-04 - model_loss: 0.1178 - val_total_loss: 0.7285 - val_cls_loss: 0.3985 - val_box_loss: 0.0055 - val_model_loss: 0.6735
Epoch 27/50
36/36 [==============================] - 8s 226ms/step - total_loss: 0.1663 - cls_loss: 0.0763 - box_loss: 6.9876e-04 - model_loss: 0.1112 - val_total_loss: 0.7476 - val_cls_loss: 0.4172 - val_box_loss: 0.0055 - val_model_loss: 0.6925
Epoch 28/50
36/36 [==============================] - 9s 233ms/step - total_loss: 0.1546 - cls_loss: 0.0672 - box_loss: 6.4627e-04 - model_loss: 0.0995 - val_total_loss: 0.7373 - val_cls_loss: 0.4090 - val_box_loss: 0.0055 - val_model_loss: 0.6823
Epoch 29/50
36/36 [==============================] - 8s 214ms/step - total_loss: 0.1697 - cls_loss: 0.0785 - box_loss: 7.2384e-04 - model_loss: 0.1146 - val_total_loss: 0.7531 - val_cls_loss: 0.4228 - val_box_loss: 0.0055 - val_model_loss: 0.6980
Epoch 30/50
36/36 [==============================] - 8s 209ms/step - total_loss: 0.1397 - cls_loss: 0.0575 - box_loss: 5.4149e-04 - model_loss: 0.0846 - val_total_loss: 0.7550 - val_cls_loss: 0.4281 - val_box_loss: 0.0054 - val_model_loss: 0.6999
Epoch 31/50
36/36 [==============================] - 7s 198ms/step - total_loss: 0.1565 - cls_loss: 0.0683 - box_loss: 6.6409e-04 - model_loss: 0.1015 - val_total_loss: 0.7491 - val_cls_loss: 0.4211 - val_box_loss: 0.0055 - val_model_loss: 0.6940
Epoch 32/50
36/36 [==============================] - 9s 241ms/step - total_loss: 0.1567 - cls_loss: 0.0673 - box_loss: 6.8615e-04 - model_loss: 0.1016 - val_total_loss: 0.7558 - val_cls_loss: 0.4182 - val_box_loss: 0.0057 - val_model_loss: 0.7008
Epoch 33/50
36/36 [==============================] - 7s 185ms/step - total_loss: 0.1513 - cls_loss: 0.0644 - box_loss: 6.3685e-04 - model_loss: 0.0962 - val_total_loss: 0.7492 - val_cls_loss: 0.4168 - val_box_loss: 0.0055 - val_model_loss: 0.6942
Epoch 34/50
36/36 [==============================] - 8s 231ms/step - total_loss: 0.1542 - cls_loss: 0.0650 - box_loss: 6.8252e-04 - model_loss: 0.0992 - val_total_loss: 0.7594 - val_cls_loss: 0.4234 - val_box_loss: 0.0056 - val_model_loss: 0.7043
Epoch 35/50
36/36 [==============================] - 8s 218ms/step - total_loss: 0.1538 - cls_loss: 0.0668 - box_loss: 6.3997e-04 - model_loss: 0.0988 - val_total_loss: 0.7570 - val_cls_loss: 0.4212 - val_box_loss: 0.0056 - val_model_loss: 0.7020
Epoch 36/50
36/36 [==============================] - 8s 225ms/step - total_loss: 0.1664 - cls_loss: 0.0734 - box_loss: 7.5846e-04 - model_loss: 0.1113 - val_total_loss: 0.7462 - val_cls_loss: 0.4087 - val_box_loss: 0.0056 - val_model_loss: 0.6911
Epoch 37/50
36/36 [==============================] - 7s 189ms/step - total_loss: 0.1573 - cls_loss: 0.0672 - box_loss: 7.0088e-04 - model_loss: 0.1023 - val_total_loss: 0.7874 - val_cls_loss: 0.4395 - val_box_loss: 0.0059 - val_model_loss: 0.7323
Epoch 38/50
36/36 [==============================] - 7s 199ms/step - total_loss: 0.1532 - cls_loss: 0.0651 - box_loss: 6.6092e-04 - model_loss: 0.0981 - val_total_loss: 0.7869 - val_cls_loss: 0.4541 - val_box_loss: 0.0056 - val_model_loss: 0.7318
Epoch 39/50
36/36 [==============================] - 8s 225ms/step - total_loss: 0.1606 - cls_loss: 0.0705 - box_loss: 7.0036e-04 - model_loss: 0.1055 - val_total_loss: 0.8196 - val_cls_loss: 0.4710 - val_box_loss: 0.0059 - val_model_loss: 0.7646
Epoch 40/50
36/36 [==============================] - 7s 190ms/step - total_loss: 0.1513 - cls_loss: 0.0626 - box_loss: 6.7240e-04 - model_loss: 0.0963 - val_total_loss: 0.7682 - val_cls_loss: 0.4303 - val_box_loss: 0.0057 - val_model_loss: 0.7131
Epoch 41/50
36/36 [==============================] - 8s 210ms/step - total_loss: 0.1246 - cls_loss: 0.0447 - box_loss: 4.9650e-04 - model_loss: 0.0696 - val_total_loss: 0.7928 - val_cls_loss: 0.4596 - val_box_loss: 0.0056 - val_model_loss: 0.7377
Epoch 42/50
36/36 [==============================] - 8s 236ms/step - total_loss: 0.1398 - cls_loss: 0.0560 - box_loss: 5.7322e-04 - model_loss: 0.0847 - val_total_loss: 0.7915 - val_cls_loss: 0.4526 - val_box_loss: 0.0057 - val_model_loss: 0.7364
Epoch 43/50
36/36 [==============================] - 7s 189ms/step - total_loss: 0.1540 - cls_loss: 0.0636 - box_loss: 7.0673e-04 - model_loss: 0.0989 - val_total_loss: 0.7714 - val_cls_loss: 0.4324 - val_box_loss: 0.0057 - val_model_loss: 0.7164
Epoch 44/50
36/36 [==============================] - 8s 217ms/step - total_loss: 0.1439 - cls_loss: 0.0594 - box_loss: 5.8977e-04 - model_loss: 0.0889 - val_total_loss: 0.7729 - val_cls_loss: 0.4394 - val_box_loss: 0.0056 - val_model_loss: 0.7179
Epoch 45/50
36/36 [==============================] - 9s 238ms/step - total_loss: 0.1295 - cls_loss: 0.0500 - box_loss: 4.8739e-04 - model_loss: 0.0744 - val_total_loss: 0.8151 - val_cls_loss: 0.4674 - val_box_loss: 0.0059 - val_model_loss: 0.7600
Epoch 46/50
36/36 [==============================] - 7s 185ms/step - total_loss: 0.1232 - cls_loss: 0.0435 - box_loss: 4.9310e-04 - model_loss: 0.0681 - val_total_loss: 0.8123 - val_cls_loss: 0.4680 - val_box_loss: 0.0058 - val_model_loss: 0.7573
Epoch 47/50
36/36 [==============================] - 9s 255ms/step - total_loss: 0.1373 - cls_loss: 0.0528 - box_loss: 5.8935e-04 - model_loss: 0.0822 - val_total_loss: 0.8149 - val_cls_loss: 0.4747 - val_box_loss: 0.0057 - val_model_loss: 0.7598
Epoch 48/50
36/36 [==============================] - 8s 221ms/step - total_loss: 0.1264 - cls_loss: 0.0471 - box_loss: 4.8371e-04 - model_loss: 0.0713 - val_total_loss: 0.8273 - val_cls_loss: 0.4877 - val_box_loss: 0.0057 - val_model_loss: 0.7723
Epoch 49/50
36/36 [==============================] - 8s 221ms/step - total_loss: 0.1150 - cls_loss: 0.0390 - box_loss: 4.1917e-04 - model_loss: 0.0600 - val_total_loss: 0.8211 - val_cls_loss: 0.4762 - val_box_loss: 0.0058 - val_model_loss: 0.7661
Epoch 50/50
36/36 [==============================] - 9s 248ms/step - total_loss: 0.1220 - cls_loss: 0.0444 - box_loss: 4.5080e-04 - model_loss: 0.0669 - val_total_loss: 0.8413 - val_cls_loss: 0.4963 - val_box_loss: 0.0058 - val_model_loss: 0.7863

"""
pattern = r"total_loss: (\d+\.\d+(?:e[+-]?\d+)?) - cls_loss: (\d+\.\d+(?:e[+-]?\d+)?) - box_loss: (\d+\.\d+(?:e[+-]?\d+)?) - model_loss: (\d+\.\d+(?:e[+-]?\d+)?) - val_total_loss: (\d+\.\d+(?:e[+-]?\d+)?) - val_cls_loss: (\d+\.\d+(?:e[+-]?\d+)?) - val_box_loss: (\d+\.\d+(?:e[+-]?\d+)?) - val_model_loss: (\d+\.\d+(?:e[+-]?\d+)?)"

matches = re.findall(pattern, txt_file)
# Convert the matched strings to float and separate them into lists
total_losses, cls_losses, box_losses, model_losses, val_total_losses, val_cls_losses, val_box_losses, val_model_losses = zip(*((float(total), float(cls), float(box), float(model), float(val_total), float(val_cls), float(val_box), float(val_model)) 
                                                           for total, cls, box, model,val_total, val_cls, val_box, val_model in matches))

epochs = range(1, len(total_losses) + 1)

plt.figure(figsize=(9,9))
plt.plot(epochs, total_losses, ('#D71939'), label="Total Loss", linewidth=2)
plt.xlabel("Epochs")
plt.ylabel("Total Loss")
plt.legend()
plt.subplot(4, 2, 2)
plt.plot(epochs, cls_losses, ('#D71939'), label="Classification Loss",linewidth=2)
plt.xlabel("Epochs")
plt.ylabel("Classification Loss")
plt.legend()

plt.subplot(4, 2, 3)
plt.plot(epochs, box_losses, ('#D71939'), label="Box Loss",linewidth=2)
plt.xlabel("Epochs")
plt.ylabel("Box Loss")
plt.legend()

plt.subplot(4, 2, 4)
plt.plot(epochs, model_losses, ('#D71939'), label="Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Model Loss")
plt.legend()

plt.subplot(4, 2, 5)
plt.plot(epochs, val_total_losses, ('#D71939'), label="Validation Total Loss")
plt.xlabel("Epochs")
plt.ylabel("Validation Total Loss")
plt.legend()

plt.subplot(4, 2, 6)
plt.plot(epochs, val_cls_losses, ('#D71939'), label="Validation Classification Loss")
plt.xlabel("Epochs")
plt.ylabel("Validation Classification Loss")
plt.legend()

plt.subplot(4, 2, 7)
plt.plot(epochs, val_box_losses, ('#D71939'), label="Validation Box Loss")
plt.xlabel("Epochs")
plt.ylabel("Validation Box Loss")
plt.legend()

plt.subplot(4, 2, 8)
plt.plot(epochs, val_model_losses, ('#D71939'), label="Validation Model Loss",linewidth=3)
plt.xlabel("Epochs")
plt.ylabel("Validation Model Loss")
plt.legend()
plt.show()

plt.tight_layout()
plt.show()

## Plotting average precision, average recall based on the IoU threshold

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt




# Given data
metrics = {
    'AP': [0.098, 0.240, 0.060, 0.022, 0.092, 0.109],
    'AR': [0.109, 0.199, 0.207, 0.113, 0.153, 0.238]
}
categories = ['IoU=0.50:0.95, all', 'IoU=0.50, all', 'IoU=0.75, all',
              'IoU=0.50:0.95, small', 'IoU=0.50:0.95, medium', 'IoU=0.50:0.95, large']
validation_loss= [0.8395382761955261, 0.49765241146087646, 0.005737063009291887, 0.7845054864883423]

# Set up the matplotlib figure
plt.figure(figsize=(14, 8))

# Draw a bar plot for AP and AR
sns.barplot(x=categories, y=metrics['AP'], color='blue', label='Average Precision (AP)')
sns.barplot(x=categories, y=metrics['AR'], color='red', label='Average Recall (AR)', alpha=0.6)

# Add a line plot for validation loss
epochs = list(range(1, len(validation_loss) + 1))
plt.plot(epochs, validation_loss, label='Validation Loss', color='green', marker='o', linewidth=2)

# Final design settings
plt.title('Average Precision, Average Recall, and Validation Loss per Category')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.ylabel('Metric Value')
plt.xlabel('Category')

# Secondary Y-axis for validation loss
plt.twinx().set_ylabel('Validation Loss')

# Adding legend
plt.legend(loc='upper left')

# Show plot
plt.show()